### Split the old file into two strands and crystal water molecules by vim
- original file: cug_rna_duplex.pdb

```
cp cug_rna_duplex.pdb ../charmm_setup/strand1.input.pdb
cp cug_rna_duplex.pdb ../charmm_setup/strand2.input.pdb
cp cug_rna_duplex.pdb ../charmm_setup/crystal.water.input.pdb
```

### Rename atomname in RNA
| Crystal | CHARMM |
|---------|--------|
| OP1     | O1P    |
| OP2     | O2P    |
| OP3     | O5T    |

#### First step
```
cat strand1.input.pdb | sed 's/OP1/O1P/' | sed 's/OP2/O2P/' > strand1.input.1.pdb
cat strand2.input.pdb | sed 's/OP1/O1P/' | sed 's/OP2/O2P/' > strand2.input.1.pdb
```

#### Second step
```
remove P O1P O2P O3P
```

#### Third step
trim the chain name after resname, and trim the last column of elements by vim

### Rename resname and atomid

In [6]:
from os import path
from pdbtools import pdb
pdb_folder = '/home/yizaochen/codes/cug_rna/charmm_setup/pdb'

In [7]:
strand1_in_pdb = path.join(pdb_folder, 'strand1.input.1.pdb')
strand1_out_pdb = path.join(pdb_folder, 'strand1.input.2.pdb')

reader = pdb.PDBReader(strand1_in_pdb)
d_resname = {'A': 'ADE', 'U': 'URA', 'G': 'GUA', 'C': 'CYT'}
for idx, atom in enumerate(reader.atomgroup):
    atom.set_resname(d_resname[atom.resname])
    atom.set_atomid(idx+1)
    
writer = pdb.PDBWriter(strand1_out_pdb, reader.atomgroup)
writer.write_pdb()

Write PDB: /home/yizaochen/codes/cug_rna/charmm_setup/pdb/strand1.input.2.pdb


In [8]:
strand2_in_pdb = path.join(pdb_folder, 'strand2.input.1.pdb')
strand2_out_pdb = path.join(pdb_folder, 'strand2.input.2.pdb')

reader = pdb.PDBReader(strand2_in_pdb)
d_resname = {'A': 'ADE', 'U': 'URA', 'G': 'GUA', 'C': 'CYT'}
for idx, atom in enumerate(reader.atomgroup):
    atom.set_resname(d_resname[atom.resname])
    atom.set_atomid(idx+1)
    
writer = pdb.PDBWriter(strand2_out_pdb, reader.atomgroup)
writer.write_pdb()

Write PDB: /home/yizaochen/codes/cug_rna/charmm_setup/pdb/strand2.input.2.pdb


### CHARMM build each molecule separately
```
charmm < strand1.inp > strand1.dat
charmm < strand2.inp > strand2.dat
```

### Rename atomname in water
| Crystal | CHARMM |
|---------|--------|
| O     | OH2    |

#### First step
```
cat crystal.water.input.pdb | sed 's/  O   /  OH2 /' > crystal.water.input.1.pdb
```
#### Second step
trim the chain name after resname, and trim the last column of elements by vim

#### Note
1. Water 22 duplicate -> remove F000's one
2. water 43 duplicate -> remove F000's one

In [5]:
in_pdb = path.join(pdb_folder, 'crystal.water.input.1.pdb')
out_pdb = path.join(pdb_folder, 'crystal.water.input.2.pdb')

reader = pdb.PDBReader(in_pdb)
for idx, atom in enumerate(reader.atomgroup):
    atom.set_resname('TIP3')
    atom.set_atomid(idx+1)
    atom.set_resid(idx+1)
    
writer = pdb.PDBWriter(out_pdb, reader.atomgroup)
writer.write_pdb()

Write PDB: /home/yizaochen/codes/cug_rna/charmm_setup/pdb/crystal.water.input.2.pdb


### Build Crystal Water By CHARMM

```
charmm < crystal_water.inp > crystal_water.dat
```

### Combine three molecules by CHARMM

```
charmm < dsrna+crywater.inp > dsrna+crywater.dat
```

### Rename resname
```
cat cug3.aa.0.pdb | sed 's/URA/RU /' | sed 's/ADE/RA /' | sed 's/GUA/RG /' | sed 's/CYT/RC /' > cug3.aa.1.pdb
```

- vim -> make 5ter and 3ter as RX5, RX3

### Rename atomname

| AMBER in CHARMM | OL15 in GROMACS |
|-------------|-----------------|
| H5'         | H5'1            |
| H5''        | H5'2            |
| H2''        | H2'1            |
| H2'         | HO'2            |

```
cat cug3.aa.1.pdb | sed "s/ H5' /H5'1 /" | sed "s/H5''/H5'2/" | sed "s/H2''/H2'1/" | sed "s/ H2' /HO'2 /" > cug3.aa.2.pdb
```

### Rename resname and atomname for Water
#### Resname
TIP3 -> SOL

#### Atomname
| AMBER in CHARMM | OL15 in GROMACS |
|-------------|-----------------|
| OH2        | OW            |
| H1        | HW1            |
| H2        | HW2            |

```
cat cug3.aa.2.pdb | sed "s/TIP3/SOL /"| sed "s/OH2/OW /" | sed "s/H1  SOL/HW1 SOL/" | sed "s/H2  SOL/HW2 SOL/" > cug3.aa.3.pdb
```

### Add TER by vim
### Upload to allostery

### VMD show and draw